### 모듈 가져오기

In [5]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/07_PJT_final_01/01_clothes_classification

/content/drive/.shortcut-targets-by-id/1GjyX6hhmkv3p8CkgvOoSe14VWUuNEGQI/07_PJT_final_01/01_clothes_classification


In [3]:
img_size = 224
channel=3

In [7]:
train_datagen = ImageDataGenerator( 
        rescale=1./255,         # 픽셀 값을 0~1 범위로 변환
        # rotation_range=40,      # 40도까지 회전
        # width_shift_range=0.2,  # 20%까지 좌우 이동
        # height_shift_range=0.2, # 20%까지 상하 이동
        # shear_range=0.2,        # 20%까지 기울임
        # zoom_range=0.2,         # 20%까지 확대
        # horizontal_flip=True,   # 좌우 뒤집기
    )
train_generator = train_datagen.flow_from_directory(
    './dataset/Train',
    batch_size=100,      
    class_mode='categorical', 
    target_size=(img_size, img_size)) 
valid_datagen = ImageDataGenerator(rescale=1./255) # 스케일링만 진행
valid_generator = valid_datagen.flow_from_directory(
    './dataset/Validation',
    batch_size=80,      
    class_mode='categorical', 
    target_size=(img_size, img_size))
test_datagen = ImageDataGenerator(rescale=1./255) # 스케일링만 진행
test_generator = test_datagen.flow_from_directory(
    './dataset/data/test',
    batch_size=10,      
    class_mode='categorical', 
    target_size=(img_size, img_size),
    shuffle=False)

Found 32426 images belonging to 4 classes.
Found 7894 images belonging to 4 classes.
Found 10 images belonging to 4 classes.


In [8]:
print('train_generator class : ', train_generator.class_indices)
print('valid_generator class : ', valid_generator.class_indices)
print('test_generator class : ', test_generator.class_indices)

train_generator class :  {'bottoms': 0, 'one-piece': 1, 'outerwear': 2, 'tops': 3}
valid_generator class :  {'bottoms': 0, 'one-piece': 1, 'outerwear': 2, 'tops': 3}
test_generator class :  {'bottoms': 0, 'one-piece': 1, 'outerwear': 2, 'tops': 3}


### 모델 학습하기 - vgg16
- imagenet 전이학습

- 출력층
   * Dense(4, activation='softmax')


In [ ]:
# 파라미터 설정
classes = len(train_generator.class_indices)
epochs = 200
batch_size = 96

In [ ]:
vgg16 = tf.keras.applications.VGG16(weights='imagenet',
                                    include_top=False, 
                                    input_shape=(224, 224, 3))
vgg16.trainable = False # 학습으로 기존 가중치 갱신을 하지 않도록 지정

In [ ]:
model = Sequential()
model.add(vgg16)
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(4, activation='softmax'))
model.summary()

In [ ]:
# 모델 환경설정
model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator,
                    epochs=100,
                    validation_data=valid_generator, 
                    callbacks = [[earlystopping, checkpointer]]) 

## 모델 이어서 학습하기

In [12]:
from tensorflow import keras
from keras.models import load_model
# model = keras.models.load_model

In [13]:
model = load_model('/content/drive/MyDrive/07_PJT_final_01/01_clothes_classification/model/VGG16_01_01-0.2435.hdf5')

In [14]:
model_name = './model/VGG16_01_{epoch:02d}-{val_loss:.4f}.hdf5'
checkpointer = ModelCheckpoint(filepath=model_name, monitor='val_loss',save_best_only=True) # 성능이 개선될떄만 저장하도록
earlystopping = EarlyStopping(monitor='val_loss', patience=5) # 5번동안 성능개선이 없으면 학습 조기종료

In [ ]:
history2 = model.fit(train_generator,
                    epochs=100,
                    validation_data=valid_generator, 
                    callbacks = [[earlystopping, checkpointer]]) 

Epoch 1/100
325/325 [==============================] - 20522s 63s/step - loss: 0.2830 - accuracy: 0.8957 - val_loss: 0.2283 - val_accuracy: 0.9173
Epoch 2/100
325/325 [==============================] - 819s 3s/step - loss: 0.2578 - accuracy: 0.9045 - val_loss: 0.2215 - val_accuracy: 0.9150
Epoch 3/100
325/325 [==============================] - 800s 2s/step - loss: 0.2314 - accuracy: 0.9160 - val_loss: 0.2313 - val_accuracy: 0.9184
Epoch 4/100
325/325 [==============================] - 806s 2s/step - loss: 0.2252 - accuracy: 0.9176 - val_loss: 0.2215 - val_accuracy: 0.9168
Epoch 5/100
325/325 [==============================] - 807s 2s/step - loss: 0.2126 - accuracy: 0.9207 - val_loss: 0.2049 - val_accuracy: 0.9258
Epoch 6/100
325/325 [==============================] - 794s 2s/step - loss: 0.1940 - accuracy: 0.9292 - val_loss: 0.2113 - val_accuracy: 0.9216
Epoch 7/100
325/325 [==============================] - 797s 2s/step - loss: 0.1992 - accuracy: 0.9266 - val_loss: 0.1929 - val_accura

In [ ]:
max(history.history['accuracy'])

In [ ]:
# 모델 저장
accuracy = round(max(history.history['accuracy'])*100,0)
model.save(f'./model/VGG16_01_model_acc{accuracy}.h5') # 최종 모델 저장

In [ ]:
model.save(f'./model/VGG16_01_model_acc{accuracy}.h5') # 최종 모델 저장

In [ ]:
plt.figure(figsize=(16,5))

# 만들어진 모델에 대해 train dataset과 validation dataset의 loss 를 그래프로 표현해 봅니다.
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc=0)

# 만들어진 모델에 대해 train dataset과 validation dataset의 accuracy 를 그래프로 표현해 봅니다.
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc=0)

In [ ]:
model.evaluate(test_generator)

In [ ]:
train_generator.class_indices

In [ ]:
# 직접 찍은 음료 이미지로 예측 결과 확인

class_dict = {0:'bottoms', 1:'one-piece', 2:'outerwear', 3:'tops'}

fig = plt.figure(figsize=(12,12))
i=0
for img in glob.glob('./dataset/test/*.jpg'):
    ax = plt.subplot(4,4,i+1) 
    test_img = cv2.imread(img)
    test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
    test_img = cv2.resize(test_img,(224,224))/255.0
    test_img_input = test_img.reshape((1,)+test_img.shape)
    pred = model.predict(test_img_input)
    pred_class = class_dict[np.argmax(pred, axis=1)[0]]
    ax.imshow(test_img)
    ax.axis('off')
    ax.set_title(f'{pred_class}',fontsize=17)
    i += 1
plt.subplots_adjust(left=0.125, bottom=0.1, right=0.9, top=0.7, wspace=0.2, hspace=0.2)
plt.show()